In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import polars as pl
import pandas as pd
import numpy as np
import tqdm

from models.t4rec_enriched import load_model, create_trainer
from torch.utils.data import DataLoader
from dataloaders.extension import ParquetDataset, dataloader_collate_fn, preprocess
from models.pci import PredictClickedInview

from models.pci_attn import AttnPredictClickedInview

from torch import nn
from tqdm import tqdm
from typing import List, Tuple

/home/scur1565/.conda/envs/transformers4rec_v2_akis/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/home/scur1565/.conda/envs/transformers4rec_v2_akis/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")
/home/scur1565/.conda/envs/transformers4rec_v2_akis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
epochs = 2
data_split = "small"
model_trained_on = "small"

per_device_train_batch_size = 2048
per_device_eval_batch_size = 2048

## Create Models

In [4]:
max_sequence_length = 20
output_dir = f"../checkpoints/{data_split}-{model_trained_on}-extension"

if model_trained_on == "small":
    model_path = "../checkpoints/enriched/small/checkpoint-226/pytorch_model.bin"
else:
    model_path = "../checkpoints/enriched/large/checkpoint-23560/pytorch_model.bin"

model = load_model(model_path, max_sequence_length=max_sequence_length, d_model=64)
model = model.cuda()
params = {}
params["max_inview"] = 100
params["user_feature_dim"] = 0
params["ovewrite_meta_model"] = True
params["use_summarizer"] = True
params["use_metadata"] = False

predict_model = AttnPredictClickedInview(model, params)

predict_model = predict_model.cuda()

trainer = create_trainer(
    model,
    output_dir,
    epochs,
    max_sequence_length,
    per_device_train_batch_size,
    per_device_eval_batch_size,
    500,
)

train_paths = [
        f"../data/advanced_ts_ebnerd_processed/train/train_data_{data_split}_0.parquet",
        f"../data/advanced_ts_ebnerd_processed/train/train_data_{data_split}_1.parquet",
        f"../data/advanced_ts_ebnerd_processed/train/train_data_{data_split}_2.parquet",
        f"../data/advanced_ts_ebnerd_processed/train/train_data_{data_split}_3.parquet",
        f"../data/advanced_ts_ebnerd_processed/train/train_data_{data_split}_4.parquet",
        f"../data/advanced_ts_ebnerd_processed/train/train_data_{data_split}_5.parquet",
        f"../data/advanced_ts_ebnerd_processed/train/train_data_{data_split}_6.parquet",
    ]

trainer.train_dataset_or_path = train_paths
merlin_dataloader = trainer.get_train_dataloader()

Projecting inputs of NextItemPredictionTask to'768' As weight tying requires the input dimension '64' to be equal to the item-id embedding dimension '768'
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


ATTENTION HEADS: 8


/home/scur1565/.conda/envs/transformers4rec_v2_akis/lib/python3.10/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


## Create Dataloaders

In [5]:
train_dataset = ParquetDataset(train_paths, ["age"], ['article_ids_inview', 'labels'], transform=preprocess)

prq_dataloader = DataLoader(
    train_dataset,
    batch_size=per_device_train_batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=dataloader_collate_fn,
)

predict_model.train()
loss_module = nn.BCEWithLogitsLoss(reduction='none')
optimizer = torch.optim.SGD(
    predict_model.parameters(), lr=1e-2, momentum=0.9
)  # Default parameters, feel free to change

## Training

In [6]:

print("Starting training")
for i in range(epochs):
    predict_model.train()
    epoch_loss = 0
    for sample in zip(tqdm(merlin_dataloader), prq_dataloader):
        # convert inputs to cuda
        history, (user, inviews, length, labels) = sample
        history = {name: value.cuda() for name, value in history[0].items()}
        inviews, length = inviews.cuda(), length.cuda()
        user = user.cuda()
        labels = labels.cuda()

        optimizer.zero_grad()
        preds = predict_model.forward(history, user, inviews, length)

        loss = loss_module(preds.squeeze(-1), labels.float())

        # Apply the mask to zero out the loss contributions from the padding elements
        mask = (torch.arange(100).expand(len(length.cpu()), 100) < length.cpu().unsqueeze(1)).cuda()
        masked_loss = loss * mask.float()

        # Calculate the mean loss over the non-padding elements
        mean_loss = masked_loss.sum() / mask.sum()
        
        epoch_loss += mean_loss
        mean_loss.backward()
        optimizer.step()
    
    print("Epoch Loss: ", epoch_loss / len(merlin_dataloader))

print("Finished training")

Starting training


100%|██████████| 113/113 [02:14<00:00,  1.19s/it]


Epoch Loss:  tensor(0.3187, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████| 113/113 [02:14<00:00,  1.19s/it]

Epoch Loss:  tensor(0.3010, device='cuda:0', grad_fn=<DivBackward0>)
Finished training


## Test

In [9]:
eval_paths = [
        f"../data/advanced_ts_ebnerd_processed/validation/validation_data_{data_split}_0.parquet",
        f"../data/advanced_ts_ebnerd_processed/validation/validation_data_{data_split}_1.parquet",
        f"../data/advanced_ts_ebnerd_processed/validation/validation_data_{data_split}_2.parquet",
        f"../data/advanced_ts_ebnerd_processed/validation/validation_data_{data_split}_3.parquet",
        f"../data/advanced_ts_ebnerd_processed/validation/validation_data_{data_split}_4.parquet",
        f"../data/advanced_ts_ebnerd_processed/validation/validation_data_{data_split}_5.parquet",
        f"../data/advanced_ts_ebnerd_processed/validation/validation_data_{data_split}_6.parquet",
    ]

validation_df = pd.read_parquet(eval_paths, engine="pyarrow")

trainer.eval_dataset_or_path = eval_paths
dlv = trainer.get_eval_dataloader()
val_dataset = ParquetDataset(eval_paths, ["age"], ['article_ids_inview', 'labels'], transform=preprocess)

val_dataset.max_inview_lenght = train_dataset.max_inview_lenght # set train max length

eval_prq_dataloader = DataLoader(val_dataset, batch_size=per_device_eval_batch_size, shuffle=False, drop_last=False, collate_fn=dataloader_collate_fn)


print(f"Ensure that Validation Dataloader is deterministic: {not dlv.shuffle}")

Ensure that Validation Dataloader is deterministic: True


### Inference

In [10]:
all_inview_scores = []
index: int = 0
predict_model.eval()
# Sample one from data loader
for sp_batch in zip(tqdm(dlv), eval_prq_dataloader):
    history, (user, inviews, length, labels) = sp_batch
    history = {name: value.cuda() for name, value in history[0].items()}
    inviews, length = inviews.cuda(), length.cuda()
    user = user.cuda()
    labels = labels.cuda()

    with torch.no_grad():
        preds = predict_model.forward(history, user, inviews, length)
    
    preds = preds.squeeze(-1).cpu().detach().numpy()
    preds = [p[:ln] for p, ln in zip(preds, length)]

    all_inview_scores.extend(preds)

  0%|          | 0/119 [00:00<?, ?it/s]

100%|██████████| 119/119 [01:28<00:00,  1.35it/s]


### Metrics

In [12]:
import pandas as pd
from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore
from typing import List, Tuple
from tqdm import tqdm

metrics = MetricEvaluator(
    labels=validation_df["labels"].to_list()[:-7],
    predictions=all_inview_scores,
    metric_functions=[AucScore(), MrrScore(), NdcgScore(k=5), NdcgScore(k=10)],
)
metrics.evaluate()

<MetricEvaluator class>: 
 {
    "auc": 0.551068365070001,
    "mrr": 0.34628988634116115,
    "ndcg@5": 0.3833610563368501,
    "ndcg@10": 0.46139431279504994
}